In [0]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [0]:
import os

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#go to working directory
cd "/content/drive/My Drive/dataset/"

/content/drive/My Drive/dataset


In [0]:
#define the variables
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)
train_dir     = 'training_set'
test_dir      = 'test_set'
num_classes = 2
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(299, 299), \
                                                    batch_size=128, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(299, 299), \
                                                    batch_size=128, class_mode='categorical')

Found 3111 images belonging to 2 classes.
Found 779 images belonging to 2 classes.


In [0]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
# Get the output layer from the pre-trained Inception V3 model
x = base_model.output

# Now, add new layers that will be trained with our data
# These layers will be randomly initialized
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Get the final Model to train
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers from the original base model so that we don't update the weights
for layer in base_model.layers:
    layer.trainable = False

In [0]:
adam = optimizers.adam(lr=0.001)
# Compile the new model
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# Setup a callback to save the best model
callbacks = [keras.callbacks.ModelCheckpoint('model.{epoch:02d}-{val_acc:.2f}.hdf5', monitor='val_acc',\
 verbose=1, save_best_only=True, mode='max', period=1)]

# Fit the data and output the history
history = model.fit_generator(train_generator, verbose=1, steps_per_epoch=len(train_generator), epochs=10, \
 validation_data=test_generator, validation_steps=len(test_generator), callbacks=callbacks)

Epoch 1/10
25/25 [==============================] - 1355s 54s/step - loss: 0.2635 - acc: 0.8976 - val_loss: 0.3985 - val_acc: 0.8511

Epoch 00001: val_acc improved from -inf to 0.85109, saving model to model.01-0.85.hdf5
Epoch 2/10
25/25 [==============================] - 1316s 53s/step - loss: 0.0955 - acc: 0.9633 - val_loss: 0.4140 - val_acc: 0.8549

Epoch 00002: val_acc improved from 0.85109 to 0.85494, saving model to model.02-0.85.hdf5
Epoch 3/10
25/25 [==============================] - 1338s 54s/step - loss: 0.0859 - acc: 0.9672 - val_loss: 0.4586 - val_acc: 0.8485

Epoch 00003: val_acc did not improve from 0.85494
Epoch 4/10
25/25 [==============================] - 1285s 51s/step - loss: 0.0580 - acc: 0.9799 - val_loss: 0.3778 - val_acc: 0.8858

Epoch 00004: val_acc improved from 0.85494 to 0.88575, saving model to model.04-0.89.hdf5
Epoch 5/10
25/25 [==============================] - 1287s 51s/step - loss: 0.0378 - acc: 0.9869 - val_loss: 0.1658 - val_acc: 0.9397

Epoch 00005: 